In [1]:
import torch
from torchvision import models
from torch import nn
from torch.utils.data import DataLoader
from LymphoMNIST.LymphoMNIST import LymphoMNIST
from torchvision import transforms
from student_qt import FilteredLymphoMNIST, get_dataloaders  

In [2]:

# Hyperparameters
params = {
    'batch_size': 16,
    'im_size': 48,  # Resize dimension used during training
    'model_checkpoint': "../checkpoint/Final_models/KD_13 October 00:13_resnet50_qt-1channel-imsize-120-48.pt"  # Path to the saved model
}

BIGGER = 48 # 64 for resnet50 and 48 for tiny model

import torchvision.transforms as T
transform_student = T.Compose([
    T.Resize((BIGGER, BIGGER)),
    T.ToTensor(),
    T.Normalize([0.4819], [0.1484]),
])

In [3]:


# Initialize datasets
labels_to_keep = [0, 1]
original_train_ds = LymphoMNIST(root='../dataset', train=True, download=True, transform=transform_student, num_classes=3)
original_test_ds = LymphoMNIST(root='../dataset', train=False, download=True, transform=transform_student, num_classes=3)

# Filter datasets
train_ds = FilteredLymphoMNIST(original_train_ds, labels_to_keep)
test_ds = FilteredLymphoMNIST(original_test_ds, labels_to_keep)

# Get dataloaders
train_dl, val_dl, test_dl = get_dataloaders(train_ds, test_ds, batch_size=params['batch_size'], num_workers=4)

Dataset already exists. Skipping download.
Dataset already exists. Skipping download.


In [4]:
def evaluate_model(model, dataloader, device):
    y_pred = []
    y_true = []

    # Set model to evaluation mode
    model.eval()

    with torch.no_grad():
        for image, target in dataloader:
            image, target = image.to(device), target.to(device)
            outputs = model(image)
            output = (torch.max(outputs, 1)[1]).data.cpu().numpy()
            y_pred.extend(output)  # Save Prediction
            target = target.data.cpu().numpy()
            y_true.extend(target)  # Save target

    return y_true, y_pred

# Resnet18

In [5]:
# Load model
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = models.resnet18()
model.conv1 = nn.Conv2d(1, model.conv1.out_channels, kernel_size=model.conv1.kernel_size, stride=model.conv1.stride, padding=model.conv1.padding, bias=False)
model.fc = nn.Linear(model.fc.in_features, len(labels_to_keep))

# Load saved weights
# model.load_state_dict(torch.load('checkpoint/KD_10 October 16:37_resnet50_resnet18-1channel-timm.pt', map_location=device))
model.load_state_dict(torch.load('../checkpoint/Final_models/KD_10 October 16:40_resnet50_resnet18-1channel-worked.pt', map_location=device))
model = model.to(device)

# Function to calculate accuracy
def calculate_accuracy(loader, model, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy

In [ ]:
from torchsummary import summary
summary(model, (1, 64, 64)) 

In [ ]:
# Calculate and print accuracies
train_acc = calculate_accuracy(train_dl, model, device)
print(f'Train Accuracy: {train_acc:.2f}%')



In [ ]:
val_acc = calculate_accuracy(val_dl, model, device)
print(f'Validation Accuracy: {val_acc:.2f}%')



In [ ]:
test_acc = calculate_accuracy(test_dl, model, device)
print(f'Test Accuracy: {test_acc:.2f}%')

## Report scores

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score

def get_metrics(y_true, y_pred):
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')
    auroc = roc_auc_score(y_true, y_pred, average='weighted', multi_class='ovr')
    accuracy = accuracy_score(y_true, y_pred)
    
    return precision, recall, f1, auroc, accuracy

# Evaluate the model on the validation set
y_true, y_pred = evaluate_model(model, test_dl, device)

# Get the metrics
precision, recall, f1, auroc, accuracy = get_metrics(y_true, y_pred)

print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1 Score: {f1:.2f}')
print(f'AUROC: {auroc:.2f}')
print(f'Accuracy: {accuracy:.2f}')

## Inference time

In [7]:
import time

# Hyperparameters
params = {
    'batch_size': 16,  # Standard batch size
    'im_size': 64,     # Resize dimension used during training
    'num_warmup_batches': 5,  # Number of warmup batches
    'num_batches': 100,  # Number of batches to measure (should cover 1000 images)
}

In [ ]:

import numpy as np
# Function to calculate inference speed
def calculate_inference_speed(loader, model, device, num_batches, warmup_batches):
    model.eval()
    total_time = 0.0
    images_processed = 0
    
    # Warm-up loop
    with torch.no_grad():
        for i, (images, _) in enumerate(loader):
            if i >= warmup_batches:
                break
            images = images.to(device)
            _ = model(images)

    # Timed inference loop
    with torch.no_grad():
        for i, (images, _) in enumerate(loader):
            if i >= num_batches:
                break
            images = images.to(device)

            start_time = time.time()  # Start timing
            _ = model(images)
            end_time = time.time()  # End timing

            # Update total time and images processed
            total_time += (end_time - start_time)
            images_processed += images.size(0)

    avg_inference_time_per_image = total_time / images_processed
    images_per_second = 1.0 / avg_inference_time_per_image
    return avg_inference_time_per_image, images_per_second

# Run the inference test for 50 iterations and collect the results
inference_times = []
inference_speeds = []

for _ in range(50):
    avg_time, throughput = calculate_inference_speed(test_dl, model, device, params['num_batches'], params['num_warmup_batches'])
    inference_times.append(avg_time)
    inference_speeds.append(throughput)

# Calculate the average and standard deviation of inference times and speeds
mean_time = np.mean(inference_times)
std_time = np.std(inference_times)
mean_speed = np.mean(inference_speeds)
std_speed = np.std(inference_speeds)

print(f'Average Inference Time per Image (over 50 runs): {mean_time:.6f} seconds ± {std_time:.6f}')
print(f'Average Inference Speed (over 50 runs): {mean_speed:.2f} images/second ± {std_speed:.2f}')

Average Inference Time per Image (over 50 runs): 0.000104 seconds ± 0.000005
Average Inference Speed (over 50 runs): 9640.56 images/second ± 407.05

## Confusion matrix

### Test data

In [ ]:
import numpy as np

# Example usage:
y_true, y_pred = evaluate_model(model, test_dl, device)

from sklearn.metrics import confusion_matrix
# Generate the confusion matrix
cm = confusion_matrix(y_true, y_pred)

accuracy = np.trace(cm) / np.sum(cm)
print(f"Accuracy: {accuracy:.2f}")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns


font_size=50
# font = {'family' : 'Arial', 'size'   : font_size}
font = {'size'   : font_size}
plt.rc('font', **font)
# plt.rc('text', usetex=True)
plt.rcParams['axes.linewidth'] = 1.50

# Define tick labels
labels = ["B Cell", "T4 Cell"]

fig, ax = plt.subplots(figsize=(10,7), dpi= 300)

original_cmap = plt.cm.get_cmap('rocket')
# create new colormap using the top 40%
new_cmap = plt.cm.colors.ListedColormap(original_cmap(np.linspace(0.7, 1, 256)))


ax = sns.heatmap(cm, annot=True, fmt="d", cmap=new_cmap, xticklabels=labels, yticklabels=labels)
# Set tick parameters
ax.tick_params(axis='both', length=5, width=0.0, color= 'gray', direction='in')


image_name = "res18_Confusion_test"
plt.savefig(image_name+ '.svg', format='svg', dpi=1200, bbox_inches='tight')
plt.savefig(image_name+ '.png', format='png', dpi=1200, bbox_inches='tight')

### Val data

In [ ]:

# Example usage:
y_true, y_pred = evaluate_model(model, val_dl, device)

from sklearn.metrics import confusion_matrix
# Generate the confusion matrix
cm = confusion_matrix(y_true, y_pred)

accuracy = np.trace(cm) / np.sum(cm)
print(f"Accuracy: {accuracy:.2f}")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np



font_size=50
# font = {'family' : 'Arial', 'size'   : font_size}
font = {'size'   : font_size}
plt.rc('font', **font)
# plt.rc('text', usetex=True)
plt.rcParams['axes.linewidth'] = 1.50

# Define tick labels
labels = ["B Cell", "T4 Cell"]

fig, ax = plt.subplots(figsize=(10,7), dpi= 300)

original_cmap = plt.cm.get_cmap('rocket')
# create new colormap using the top 40%
new_cmap = plt.cm.colors.ListedColormap(original_cmap(np.linspace(0.7, 1, 256)))


ax = sns.heatmap(cm, annot=True, fmt="d", cmap=new_cmap, xticklabels=labels, yticklabels=labels)
# Set tick parameters
ax.tick_params(axis='both', length=5, width=0.0, color= 'gray', direction='in')


image_name = "res18_Confusion_val"
plt.savefig(image_name+ '.svg', format='svg', dpi=1200, bbox_inches='tight')
plt.savefig(image_name+ '.png', format='png', dpi=1200, bbox_inches='tight')

# Tiny model

In [6]:
# Load model
device = 'cuda' if torch.cuda.is_available() else 'cpu'

class QuantizedCNN(nn.Module):
    def __init__(self, num_classes=2, input_size=(1, 28, 28)):
        super(QuantizedCNN, self).__init__()
        self.num_classes = num_classes
        self.features = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0),
            nn.Conv2d(16, 16, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        )
        with torch.no_grad():
            dummy_input = torch.zeros(1, *input_size)
            dummy_output = self.features(dummy_input)
            num_ftrs = dummy_output.numel() // dummy_output.size(0)
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Dropout(0.5),
            nn.Linear(num_ftrs, num_classes),
            nn.Softmax(dim=1)
        )
    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x
    
    
# Load saved weights
model = QuantizedCNN(num_classes=2, input_size=(1, BIGGER, BIGGER)).to(device)

# model.load_state_dict(torch.load('checkpoint/KD_10 October 16:38_resnet50_qt-1channel-worked.pt', map_location=device))
model.load_state_dict(torch.load(params['model_checkpoint'], map_location=device))

model = model.to(device)

# Function to calculate accuracy
def calculate_accuracy(loader, model, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy

In [ ]:
from torchsummary import summary
summary(model, (1, 48, 48))

In [8]:
# Calculate and print accuracies
train_acc = calculate_accuracy(train_dl, model, device)
print(f'Train Accuracy: {train_acc:.2f}%')


Train Accuracy: 90.83%


In [9]:

val_acc = calculate_accuracy(val_dl, model, device)
print(f'Validation Accuracy: {val_acc:.2f}%')



Validation Accuracy: 89.73%


In [ ]:
test_acc = calculate_accuracy(test_dl, model, device)
print(f'Test Accuracy: {test_acc:.2f}%')

### Report scores

In [10]:
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score

def get_metrics(y_true, y_pred):
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')
    auroc = roc_auc_score(y_true, y_pred, average='weighted', multi_class='ovr')
    accuracy = accuracy_score(y_true, y_pred)
    
    return precision, recall, f1, auroc, accuracy

# Evaluate the model on the validation set
y_true, y_pred = evaluate_model(model, test_dl, device)

# Get the metrics
precision, recall, f1, auroc, accuracy = get_metrics(y_true, y_pred)

print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1 Score: {f1:.2f}')
print(f'AUROC: {auroc:.2f}')
print(f'Accuracy: {accuracy:.2f}')

Precision: 0.90
Recall: 0.89
F1 Score: 0.89
AUROC: 0.90
Accuracy: 0.89


## Inference time

In [8]:
import time

# Hyperparameters
params = {
    'batch_size': 16,  # Standard batch size
    'im_size': 48,     # Resize dimension used during training
    'num_warmup_batches': 5,  # Number of warmup batches
    'num_batches': 100,  # Number of batches to measure (should cover 1000 images)
}

In [ ]:

import numpy as np
# Function to calculate inference speed
def calculate_inference_speed(loader, model, device, num_batches, warmup_batches):
    model.eval()
    total_time = 0.0
    images_processed = 0
    
    # Warm-up loop
    with torch.no_grad():
        for i, (images, _) in enumerate(loader):
            if i >= warmup_batches:
                break
            images = images.to(device)
            _ = model(images)

    # Timed inference loop
    with torch.no_grad():
        for i, (images, _) in enumerate(loader):
            if i >= num_batches:
                break
            images = images.to(device)

            start_time = time.time()  # Start timing
            _ = model(images)
            end_time = time.time()  # End timing

            # Update total time and images processed
            total_time += (end_time - start_time)
            images_processed += images.size(0)

    avg_inference_time_per_image = total_time / images_processed
    images_per_second = 1.0 / avg_inference_time_per_image
    return avg_inference_time_per_image, images_per_second

# Run the inference test for 50 iterations and collect the results
inference_times = []
inference_speeds = []

for _ in range(50):
    avg_time, throughput = calculate_inference_speed(test_dl, model, device, params['num_batches'], params['num_warmup_batches'])
    inference_times.append(avg_time)
    inference_speeds.append(throughput)

# Calculate the average and standard deviation of inference times and speeds
mean_time = np.mean(inference_times)
std_time = np.std(inference_times)
mean_speed = np.mean(inference_speeds)
std_speed = np.std(inference_speeds)

print(f'Average Inference Time per Image (over 50 runs): {mean_time:.6f} seconds ± {std_time:.6f}')
print(f'Average Inference Speed (over 50 runs): {mean_speed:.2f} images/second ± {std_speed:.2f}')

Average Inference Time per Image (over 50 runs): 0.000015 seconds ± 0.000001
Average Inference Speed (over 50 runs): 66891.07 images/second ± 5215.63

## Confusion matrix

### Test data

In [ ]:
import numpy as np

# Example usage:
y_true, y_pred = evaluate_model(model, test_dl, device)

from sklearn.metrics import confusion_matrix
# Generate the confusion matrix
cm = confusion_matrix(y_true, y_pred)

accuracy = np.trace(cm) / np.sum(cm)
print(f"Accuracy: {accuracy:.2f}")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns





font_size=50
# font = {'family' : 'Arial', 'size'   : font_size}
font = {'size'   : font_size}
plt.rc('font', **font)
# plt.rc('text', usetex=True)
plt.rcParams['axes.linewidth'] = 1.50

# Define tick labels
labels = ["B Cell", "T4 Cell"]

fig, ax = plt.subplots(figsize=(10,7), dpi= 300)

original_cmap = plt.cm.get_cmap('rocket')
# create new colormap using the top 40%
new_cmap = plt.cm.colors.ListedColormap(original_cmap(np.linspace(0.7, 1, 256)))


ax = sns.heatmap(cm, annot=True, fmt="d", cmap=new_cmap, xticklabels=labels, yticklabels=labels)
# Set tick parameters
ax.tick_params(axis='both', length=5, width=0.0, color= 'gray', direction='in')


image_name = "Tiny_Confusion_test"
plt.savefig(image_name+ '.svg', format='svg', dpi=1200, bbox_inches='tight')
plt.savefig(image_name+ '.png', format='png', dpi=1200, bbox_inches='tight')

### Val data

In [ ]:

# Example usage:
y_true, y_pred = evaluate_model(model, val_dl, device)

from sklearn.metrics import confusion_matrix
# Generate the confusion matrix
cm = confusion_matrix(y_true, y_pred)

accuracy = np.trace(cm) / np.sum(cm)
print(f"Accuracy: {accuracy:.2f}")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np



font_size=50
# font = {'family' : 'Arial', 'size'   : font_size}
font = {'size'   : font_size}
plt.rc('font', **font)
# plt.rc('text', usetex=True)
plt.rcParams['axes.linewidth'] = 1.50

# Define tick labels
labels = ["B Cell", "T4 Cell"]

fig, ax = plt.subplots(figsize=(10,7), dpi= 300)

original_cmap = plt.cm.get_cmap('rocket')
# create new colormap using the top 40%
new_cmap = plt.cm.colors.ListedColormap(original_cmap(np.linspace(0.7, 1, 256)))


ax = sns.heatmap(cm, annot=True, fmt="d", cmap=new_cmap, xticklabels=labels, yticklabels=labels)
# Set tick parameters
ax.tick_params(axis='both', length=5, width=0.0, color= 'gray', direction='in')


image_name = "Tiny_Confusion_val"
plt.savefig(image_name+ '.svg', format='svg', dpi=1200, bbox_inches='tight')
plt.savefig(image_name+ '.png', format='png', dpi=1200, bbox_inches='tight')

## TSNE

In [ ]:
print(model)

In [ ]:
import numpy as np

# defining hook to access output from intermidiate layer
def get_features(name):
    def hook(model, input, output):
        features[name] = output.detach()
    return hook

# registerning the hook. different layer can be accessed by changing
# fc[3] parameter. for details 
# https://kozodoi.me/python/deep%20learning/pytorch/tutorial/2021/05/27/extracting-features.html

# print(model)
model.classifier[2].register_forward_hook(get_features('feats'))
# cnn.conv1.register_forward_hook(get_features('feats'))


y_pred = []
y_true = []
FEATS = [] # extracted feats will be saved here
# placeholder for batch features
features = {}
# targets = []

with torch.no_grad():
    for image, targets in test_dl:
        image, targets = image.to(device), targets.to(device)
        outputs = model(image)
        preds = (torch.max(outputs, 1)[1]).data.cpu().numpy()
        # y_pred.extend(output) # Save Prediction
        # target = target.data.cpu().numpy()
        
        y_true.extend(targets.cpu().numpy())
        y_pred.extend(preds)
        
        
        # targets.extend(target) # Save target
        FEATS.extend((features['feats'].cpu().numpy())) # Save feature
        
y_pred, y_true = np.array(y_pred).reshape(-1,1), np.array(y_true).reshape(-1,1)
FEATS = np.array(FEATS)
print(f' Feature extracted with shape: (m, C, H, W) = {np.array(FEATS).shape}')

In [ ]:
from sklearn.manifold import TSNE

# Load the dataset and perform t-SNE embedding
tsne_proj = TSNE(n_components=2, perplexity= 15,n_iter=400).fit_transform(FEATS.reshape(FEATS.shape[0], -1))



### recreate

In [ ]:
# save the t-SNE projection
import pandas as pd
df = pd.DataFrame(np.concatenate((tsne_proj, y_pred, y_true), axis=1))
df.to_csv('feats.csv', index=False)

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('feats.csv')

# FEATS = last two colimns
tsne_proj = df.iloc[:, -4:-2].values
y_pred, y_true = df.iloc[:, -2].values, df.iloc[:, -1].values
print(f' TSNE shape: (m, n) = {np.array(tsne_proj).shape}')

### Visualize

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib
import numpy as np

colors2 = ['#1f77b4', '#ff7f0e'] # matplotlib default color cycle
font_size=50
# font = {'family' : 'Arial', 'size'   : font_size}
font = {'size'   : font_size}
plt.rc('font', **font)
plt.rc('text', usetex=True)
plt.rcParams['axes.linewidth'] = 1.50


fig, ax = plt.subplots(figsize=(20,15), dpi= 300)

# Rename the ticks
plot_labels = ['B Cell', 'T4 Cell']  

for lab in range(2):
    indices = np.array(y_pred)==lab
    indices_flat = indices.flatten()

    sc = ax.scatter(tsne_proj[indices_flat,0],tsne_proj[indices_flat,1], s=150, color=colors2[lab],  label = plot_labels[lab], alpha=0.7)


# plt.legend(fontsize=35)
plt.axis('off')

image_name = "TSNE_visu"
plt.savefig(image_name+ '.svg', format='svg', dpi=1200, bbox_inches='tight')
plt.savefig(image_name+ '.png', format='png', dpi=1200, bbox_inches='tight')

# 

## ROC

In [14]:
model.eval()  # set the model to evaluation mode

# Variable for model predictions and true labels
y_pred = []
y_true = []

roc_data = {"tiny_test":[], 'tiny_val':[]}


with torch.no_grad():
    for image, target in test_dl:
        image, target = image.to(device), target.to(device)
        outputs = model(image)
        probs = torch.nn.functional.softmax(outputs, dim=1)
        # Get the probability of the positive class (class 1)
        pos_probs = probs[:, 1]
        y_pred.extend(pos_probs.cpu().numpy())  # Save predicted probabilities
        target = target.data.cpu().numpy()
        y_true.extend(target)  # Save actual labels
        
        



from sklearn.metrics import roc_curve, auc
# Compute ROC curve
fpr, tpr, thresholds = roc_curve(y_true, y_pred)
roc_auc = auc(fpr, tpr)
roc_data['tiny_test'] = [fpr, tpr, roc_auc]


with torch.no_grad():
    for image, target in val_dl:
        image, target = image.to(device), target.to(device)
        outputs = model(image)
        probs = torch.nn.functional.softmax(outputs, dim=1)
        # Get the probability of the positive class (class 1)
        pos_probs = probs[:, 1]
        y_pred.extend(pos_probs.cpu().numpy())  # Save predicted probabilities
        target = target.data.cpu().numpy()
        y_true.extend(target)  # Save actual labels
        
        



from sklearn.metrics import roc_curve, auc
# Compute ROC curve
fpr, tpr, thresholds = roc_curve(y_true, y_pred)
roc_auc = auc(fpr, tpr)
roc_data['tiny_val'] = [fpr, tpr, roc_auc]


In [16]:
import json
import numpy as np  

# File path for saving the JSON
file_path = "tiny_roc.json"

# Convert numpy arrays to lists
roc_data_serializable = {
    key: [arr.tolist() if isinstance(arr, np.ndarray) else arr for arr in value]
    for key, value in roc_data.items()
}

# Saving the dictionary to a JSON file
with open(file_path, 'w') as json_file:
    json.dump(roc_data_serializable, json_file)

In [ ]:
import matplotlib.pyplot as plt

# Plot ROC curve
plt.figure()
lw = 3  # line width
multiplier = 0.9
colors3 = ['#2ba77b', '#e9a001', '#2274b2']

fig, ax = plt.subplots(figsize=(4,4), dpi= 1200)

fpr, tpr, roc_auc = roc_data['test']

# fpr, tpr, roc_auc = np.array(fpr)*multiplier, np.array(tpr)*multiplier, roc_auc*multiplier

plt.plot(fpr, tpr, '-', color=colors3[0], lw=lw, label='Test area = %0.2f' % roc_auc)

fpr, tpr, roc_auc = roc_data['val']
# fpr, tpr, roc_auc = np.array(fpr)*multiplier, np.array(tpr)*multiplier, roc_auc*multiplier
plt.plot(fpr, tpr, '--', color=colors3[1], lw=lw, label='Val area = %0.2f' % roc_auc)

plt.plot([0, 1], [0, 1], color=colors3[2], lw=lw, linestyle='--')
# plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.01])



# plt.legend(loc="lower right")
# plt.show()



font_size=20
# font = {'family' : 'Arial', 'size'   : font_size}
font = {'size'   : font_size}
plt.rc('font', **font)
# plt.rc('text', usetex=True)
plt.rcParams['axes.linewidth'] = 1.50



# Set tick parameters
ax.tick_params(axis='both', length=5, width=1, color= 'gray', direction='in')

# Turn off the top and right spines
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Set the color of the axes
ax.spines['bottom'].set_color('gray')
ax.spines['left'].set_color('gray')


plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
# plt.title('Receiver Operating Characteristic example')
plt.legend(fontsize=font_size*0.65, loc="lower right")


# Modify x and y tick labels
xticks = ax.get_xticks()
yticks = ax.get_yticks()

# Remove the first tick
# if len(xticks) > 1:
#     ax.set_xticks(xticks[1:])
# if len(yticks) > 1:
#     ax.set_yticks(yticks[1:])

# # plt.show()
image_name = "CNN_ROC"
plt.savefig(image_name+ '.svg', format='svg', dpi=1200, bbox_inches='tight')
plt.savefig(image_name+ '.png', format='png', dpi=1200, bbox_inches='tight')


In [ ]:
import json
import matplotlib.pyplot as plt
import numpy as np

# Load the JSON files
with open('tiny_roc.json', 'r') as f:
    tiny_roc_data = json.load(f)

with open('res18_roc.json', 'r') as f:
    res18_roc_data = json.load(f)

# Extract the data
fpr_res18_test, tpr_res18_test, auc_res18_test = res18_roc_data['res18_test']
fpr_res18_val, tpr_res18_val, auc_res18_val = res18_roc_data['res18_val']

fpr_tiny_test, tpr_tiny_test, auc_tiny_test = tiny_roc_data['tiny_test']
fpr_tiny_val, tpr_tiny_val, auc_tiny_val = tiny_roc_data['tiny_val']

# Plot ROC curve
plt.figure()
lw = 3  # line width
multiplier = 0.9
# colors3 = ['#7bbbe0', '#2274b2', '#818082', '#545253', '#2ba77b']
colors3 = ['#7bbbe0', '#2274b2', '#f3df3d', '#e9a001', '#2ba77b']
 

fig, ax = plt.subplots(figsize=(4,3.5), dpi=1200)

# Plot res18 test
plt.plot(fpr_res18_test, tpr_res18_test, '-', color=colors3[0], lw=lw, label='S1 Test area = %0.2f' % auc_res18_test)
# Plot res18 validation
plt.plot(fpr_res18_val, tpr_res18_val, '--', color=colors3[1], lw=lw, label='S1 Val area  = %0.2f' % auc_res18_val)

# Plot tiny test
plt.plot(fpr_tiny_test, tpr_tiny_test, '-', color=colors3[2], lw=lw, label='S2 Test area = %0.2f' % auc_tiny_test)
# Plot tiny validation
plt.plot(fpr_tiny_val, tpr_tiny_val, '--', color=colors3[3], lw=lw, label='S2 Val area  = %0.2f' % auc_tiny_val)

# Diagonal line
plt.plot([0, 1], [0, 1], color=colors3[4], lw=lw, linestyle='--')

plt.ylim([0.0, 1.01])

font_size = 20
font = {'size': font_size}
plt.rc('font', **font)

# Customize axes
ax.tick_params(axis='both', length=5, width=1, color='gray', direction='in')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_color('gray')
ax.spines['left'].set_color('gray')

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(fontsize=font_size * 0.5, loc="lower right")

# Save the plot
# # plt.show()
image_name = "Student_ROC"
plt.savefig(image_name+ '.svg', format='svg', dpi=1200, bbox_inches='tight')
plt.savefig(image_name+ '.png', format='png', dpi=1200, bbox_inches='tight')


In [ ]:
import matplotlib.pyplot as plt

# Define the 10 colors
colors10 = ['#7bbbe0', '#f3df3d', '#2ba77b', '#2274b2', '#e9a001','#818082', '#545253', '#8d2db1', '#d17aad', '#d4580d']

# Create a figure and axis
fig, ax = plt.subplots(figsize=(8, 6))

# Plot 10 bars with the specified colors
for i, color in enumerate(colors10):
    ax.bar(i, 1, color=color, label=f'{color}')

# Add labels and title
ax.set_xticks(range(10))
ax.set_xticklabels([f'Color {i+1}' for i in range(10)], rotation=45, ha='right')
ax.set_title("10 Color Bars")
ax.set_yticks([])  # Hide y-axis ticks

# Add a legend to display the color codes
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))

# Show the plot
plt.tight_layout()
plt.show()


## parameter size

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes, mark_inset

colors3 = ['#2ba77b', '#e9a001', '#2274b2']

def plot_model_comparison(teacher_params, student1_params, student2_params,
                          base_radius=0.5, inset_position=None, zoom_factor=100, save_path=None):

    font_size = 10
    font = {'size': font_size}
    plt.rc('font', **font)
    plt.rcParams['axes.linewidth'] = 1.50

    # Calculate the multiplying factors based on parameters
    total_params = teacher_params + student1_params + student2_params
    teacher_factor = teacher_params / total_params
    student1_factor = student1_params / total_params
    student2_factor = student2_params / total_params

    # Normalize radii
    teacher_radius = base_radius * teacher_factor
    student1_radius = base_radius * student1_factor
    student2_radius = base_radius * student2_factor

    # Create a figure and a set of subplots
    fig, ax = plt.subplots(figsize=(5, 4.5), dpi=1200)

    # Determine the bottom baseline for the circles
    baseline_y = 0.1 * base_radius  # Margin around the plot

    # Calculate positions to minimize white space and ensure visibility of all three circles
    distance = 0.1 * base_radius  # Small gap between circles
    teacher_center = (teacher_radius + distance, baseline_y + teacher_radius)
    student1_center = (teacher_center[0] + teacher_radius + student1_radius + distance,
                       baseline_y + student1_radius)
    student2_center = (student1_center[0] + student1_radius + student2_radius + distance,
                       baseline_y + student2_radius)

    # Add circles for teacher and students
    teacher_circle = plt.Circle(teacher_center, teacher_radius, edgecolor='none',
                                facecolor=colors3[0], label='Teacher', alpha=0.5)
    student1_circle = plt.Circle(student1_center, student1_radius, edgecolor='none',
                                 facecolor=colors3[1], label='Student 1', alpha=0.5)
    student2_circle = plt.Circle(student2_center, student2_radius, edgecolor='none',
                                 facecolor=colors3[2], label='Student 2', alpha=0.5)

    # Add circles to the main plot
    ax.add_artist(teacher_circle)
    ax.add_artist(student1_circle)
    ax.add_artist(student2_circle)

    # Setting the limits of the main plot
    max_radius = max(teacher_radius, student1_radius, student2_radius)
    ax.set_xlim(0, student2_center[0] + student2_radius + 0.1 * base_radius)
    ax.set_ylim(0, baseline_y + 2 * max_radius + 0.1 * base_radius)
    ax.set_aspect('equal', 'box')

    # Remove axis from the main plot
    ax.axis('off')

    # Adding legend
    # plt.legend([teacher_circle, student1_circle, student2_circle],
    #            ['Teacher', 'Student 1', 'Student 2'])

    # Create a zoomed inset axes with a border
    if inset_position is not None:
        # inset_position should be [x0, y0, width, height] in axes coordinates (0 to 1)
        axins = zoomed_inset_axes(ax, zoom=zoom_factor, loc='right', borderpad=0,
                                  bbox_to_anchor=inset_position,
                                  bbox_transform=ax.transAxes)
    else:
        # Default position if inset_position is not provided
        axins = zoomed_inset_axes(ax, zoom=zoom_factor, loc='center right', borderpad=1)

    # Add circles to the inset plot
    teacher_circle_ins = plt.Circle(teacher_center, teacher_radius, edgecolor='none',
                                    facecolor=colors3[0], alpha=0.5)
    student1_circle_ins = plt.Circle(student1_center, student1_radius, edgecolor='none',
                                     facecolor=colors3[1], alpha=0.5)
    student2_circle_ins = plt.Circle(student2_center, student2_radius, edgecolor='none',
                                     facecolor=colors3[2], alpha=0.5)

    axins.add_patch(teacher_circle_ins)
    axins.add_patch(student1_circle_ins)
    axins.add_patch(student2_circle_ins)

    # Set the limits of the inset plot to focus on the Student 2 circle
    x1 = student2_center[0] - 2 * student2_radius
    x2 = student2_center[0] + 2 * student2_radius
    y1 = student2_center[1] - 2 * student2_radius
    y2 = student2_center[1] + 2 * student2_radius

    axins.set_xlim(x1, x2)
    axins.set_ylim(y1, y2)
    axins.set_aspect('equal', 'box')

    # Remove tick labels from the inset plot
    axins.set_xticks([])
    axins.set_yticks([])
    axins.set_xticklabels([])
    axins.set_yticklabels([])

    # Add a border around the inset axes
    for spine in axins.spines.values():
        spine.set_edgecolor('black')
        spine.set_linewidth(1)

    # Indicate the zoomed area on the main plot
    mark_inset(ax, axins, loc1=2, loc2=4, fc="none", ec="0.5")

    # Save the figure if a save path is provided
    if save_path:
        plt.savefig(save_path + '.svg', format='svg', dpi=1200, bbox_inches='tight')
        plt.savefig(save_path + '.png', format='png', dpi=1200, bbox_inches='tight')

    # Display the plot
    plt.show()

# Example usage
T_pram = 23512130
s1_pram = 11171266
s2_pram = 5682

# Specify the inset position [x0, y0, width, height] in axes coordinates (0 to 1)
inset_position = [0.76, 0.1, 0.25, 0.25]  # Adjust these values as needed

# Call the function with the desired zoom factor
plot_model_comparison(T_pram, s1_pram, s2_pram, inset_position=inset_position, zoom_factor=200, save_path="params")


## Inference time

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Data for inference speed
models = ['Teacher', 'Student 1', 'Student 2']
inference_speed = [3979.52, 9640.56, 66891.07]  # Images per second
inference_speed_std = [114.05, 407.05, 5215.63]  # Standard deviation

# Plot settings
font_size = 20
font = {'size': font_size}
plt.rc('font', **font)
plt.rcParams['axes.linewidth'] = 1.50

# Colors for the bars
colors3 = ['#2ba77b', '#e9a001', '#2274b2']

# Width of the bars
bar_width = 0.6

# Create the plot
fig, ax = plt.subplots(figsize=(2, 3.1), dpi=100)

# Plot the bar chart
bars = ax.bar(models, inference_speed, yerr=inference_speed_std, align='center', alpha=0.5, 
              edgecolor='none', linewidth=0, error_kw={'elinewidth': 0.5}, width=bar_width, capsize=3)

# Color each bar
for i, bar in enumerate(bars):
    bar.set_color(colors3[i])

# Customize tick parameters
ax.tick_params(axis='both', length=10, width=0.0, color='gray', direction='in')

# Set title and adjust y-ticks
# ax.title.set_text('Inference Speed (Images/Second)')
# plt.yticks(fontsize=12)
plt.xticks(rotation=90)
plt.ylim([0, 80000])  # Adjust the limit for better visibility
# Set y-axis to scientific notation
ax.ticklabel_format(style='scientific', axis='y', scilimits=(0, 0))

# Save the plot
image_name = "./inference_speed"
plt.savefig(image_name + '.svg', format='svg', dpi=1200, bbox_inches='tight')
plt.savefig(image_name + '.png', format='png', dpi=1200, bbox_inches='tight')

# # Show the plot
# plt.show()
